# 目的
以下を計算する
- 外部データセット(Fake_data_1850_218.json)のラベル数

In [10]:
import polars as pl
import matplotlib.pyplot as plt
import ast

import itertools
from collections import Counter
from spacy import displacy

In [3]:
fake_data = pl.read_json("../data/Fake_data_1850_218.json")

In [4]:
fake_data

document,full_text,tokens,trailing_whitespace,labels
i64,str,list[str],list[bool],list[str]
0,"""Leveraging Lea…","[""Leveraging"", ""Lean"", … ""Richard""]","[true, true, … false]","[""O"", ""O"", … ""B-NAME_STUDENT""]"
1,"""From Chaos to …","[""From"", ""Chaos"", … ""Katherine""]","[true, true, … false]","[""O"", ""O"", … ""B-NAME_STUDENT""]"
2,"""Leveraging Lea…","[""Leveraging"", ""Lean"", … ""Robert""]","[true, true, … false]","[""O"", ""O"", … ""B-NAME_STUDENT""]"
3,"""""Empowering Vu…","["""""", ""Empowering"", … "" ""]","[false, true, … false]","[""O"", ""O"", … ""I-NAME_STUDENT""]"
4,"""Rehabilitating…","[""Rehabilitating"", ""Hope"", … ""King""]","[true, false, … false]","[""O"", ""O"", … ""I-URL_PERSONAL""]"
5,"""Leveraging Lea…","[""Leveraging"", ""Lean"", … ""Kennedy""]","[true, true, … false]","[""O"", ""O"", … ""I-URL_PERSONAL""]"
6,"""Streamlining H…","[""Streamlining"", ""Healthcare"", … ""Ramos""]","[true, true, … false]","[""O"", ""O"", … ""I-NAME_STUDENT""]"
7,"""Navigating Com…","[""Navigating"", ""Complex"", … ""lindawest@gmail.com""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
8,"""Leveraging Tec…","[""Leveraging"", ""Technology"", … ""DDS""]","[true, true, … false]","[""O"", ""O"", … ""I-NAME_STUDENT""]"


# Visualization

In [15]:
import spacy
from spacy.tokens import Span, Doc
from spacy import displacy

# https://matplotlib.org/2.0.2/examples/color/named_colors.html
options = {
    "colors": {
        "B-NAME_STUDENT": "aqua",
        "I-NAME_STUDENT": "aliceblue",
        "B-EMAIL": "forestgreen",
        "I-EMAIL": "greenyellow",
        "B-USERNAME": "hotpink",
        "I-USERNAME": "lightpink",
        "B-ID_NUM": "purple",
        "I-ID_NUM": "rebeccapurple",
        "B-PHONE_NUM": "red",
        "I-PHONE_NUM": "salmon",
        "B-URL_PERSONAL": "slategrey",
        "I-URL_PERSONAL": "silver",
        "B-STREET_ADDRESS": "brown",
        "I-STREET_ADDRESS": "chocolate",
    }
}


class PiiLabelRenderer:
    def __init__(
        self,
        documents: list[str],
        tokens: list[str],
        full_texts: list[str],
        labels: list[str],
        options: dict[str, dict[str, str]],
    ) -> None:
        self.documents = documents
        self.tokens = tokens
        self.full_texts = full_texts
        self.labels = labels
        self.options = options
        self.nlp = spacy.blank("en")

    def get_original_text(
        self, render_idx: int | None = None, documents_id: int | None = None
    ) -> str:
        if render_idx and documents_id:
            raise Exception("Don't Use Both render_idx and documents_id")
        if documents_id:
            render_idx = self.documents.index(documents_id)

        return self.full_texts[render_idx]

    def render(
        self, render_idx: int | None = None, documents_id: int | None = None
    ) -> None:
        if render_idx and documents_id:
            raise Exception("Don't Use Both render_idx and documents_id")
        if documents_id and render_idx is None:
            render_idx = self.documents.index(documents_id)
        if documents_id is None and render_idx:
            documents_id = self.documents[render_idx]
        if documents_id is None and render_idx is None:
            raise Exception("Please specify either render_idx or documents_id")

        print(f"render_idx: {render_idx}, documents_id: {documents_id}")

        doc = self.nlp(self.full_texts[render_idx])
        doc.ents = [
            Span(doc, idx, idx + 1, label=label)
            for idx, label in enumerate(self.labels[render_idx])
            if label != "O"
        ]

        displacy.render(doc, style="ent", jupyter=True, options=options)

In [12]:
# documents = pii_dataset_fixed.get_column("document").to_list()
# tokens = pii_dataset_fixed.get_column("tokens").to_list()
# full_texts = pii_dataset_fixed.get_column("text").to_list()
# labels = pii_dataset_fixed.get_column("labels").to_list()

documents = fake_data.get_column("document").to_list()
tokens = fake_data.get_column("tokens").to_list()
full_texts = fake_data.get_column("full_text").to_list()
labels = fake_data.get_column("labels").to_list()

In [17]:
PLR = PiiLabelRenderer(
    documents=documents,
    tokens=tokens,
    labels=labels,
    options=options,
    full_texts=full_texts,
)

In [27]:
PLR.render(render_idx=9)

render_idx: 9, documents_id: 9
